In [1]:

from configparser import ConfigParser
import psycopg2
from typing import Dict


def load_connection_info(
    ini_filename: str
) -> Dict[str, str]:
    parser = ConfigParser()
    parser.read(ini_filename)
    # Create a dictionary of the variables stored under the "postgresql" section of the .ini
    conn_info = {param[0]: param[1] for param in parser.items("postgresql")}
    return conn_info


def create_db(
    conn_info: Dict[str, str],
) -> None:
    # Connect just to PostgreSQL with the user loaded from the .ini file
    psql_connection_string = f"user={conn_info['user']} password={conn_info['password']}"
    conn = psycopg2.connect(psql_connection_string)
    cur = conn.cursor()

    # "CREATE DATABASE" requires automatic commits
    conn.autocommit = True
    sql_query = f"CREATE DATABASE {conn_info['database']}"

    try:
        cur.execute(sql_query)
    except Exception as e:
        print(f"{type(e).__name__}: {e}")
        print(f"Query: {cur.query}")
        cur.close()
    else:
        # Revert autocommit settings
        conn.autocommit = False


def create_table(
    sql_query: str, 
    conn: psycopg2.extensions.connection, 
    cur: psycopg2.extensions.cursor
) -> None:
    try:
        # Execute the table creation query
        cur.execute(sql_query)
    except Exception as e:
        print(f"{type(e).__name__}: {e}")
        print(f"Query: {cur.query}")
        conn.rollback()
        cur.close()
    else:
        # To take effect, changes need be committed to the database
        conn.commit()


if __name__ == "__main__":
    # host, database, user, password
    conn_info = load_connection_info("db.ini")

    # Create the desired database
    create_db(conn_info)

    # Connect to the database created
    connection = psycopg2.connect(**conn_info)
    cursor = connection.cursor()
 
    # Create the "demographic_info" table
    demographic_info = """
        CREATE TABLE demographic_info (
            demo_key INTEGER PRIMARY KEY,
            ever_married TEXT,
            work_type TEXT,
            residence TEXT
        )
    """
    create_table(demographic_info, connection, cursor)
    


    # Create the "health_attribute" table
    health_attribute = """
        CREATE TABLE health_attribute (
            health_key INTEGER PRIMARY KEY,
            hypertension INTEGER,
            heart_disease INTEGER,
            smoking_status TEXT,
            avg_glucose DOUBLE PRECISION,
            BMI DOUBLE PRECISION
        )
    """
    create_table(health_attribute, connection, cursor)

    # Create the "personal_attributes" table
    personal_attributes  = """
        CREATE TABLE personal_attributes (
            personal_key INTEGER PRIMARY KEY,
            gender TEXT,
            age DOUBLE PRECISION
        )
    """
    create_table(personal_attributes, connection, cursor)

    # Create the "fact_table" table
    fact_table = """
        CREATE TABLE fact_table (
            stroke INTEGER,
            Total_strokes INTEGER,
            Paitent_count INTEGER,
            Number_smokers INTEGER,
            Bmi_level INTEGER,
            demo_key INTEGER REFERENCES demographic_info(demo_key),
            health_key INTEGER REFERENCES health_attribute(health_key),
            personal_key INTEGER REFERENCES personal_attributes(personal_key)
        )
    """
    create_table(fact_table, connection, cursor)

    # Close all connections to the database
    connection.close()
    cursor.close()

DuplicateDatabase: database "stroke_db" already exists

Query: b'CREATE DATABASE stroke_db'
DuplicateTable: relation "demographic_info" already exists

Query: b'\n        CREATE TABLE demographic_info (\n            demo_key INTEGER PRIMARY KEY,\n            ever_married TEXT,\n            work_type TEXT,\n            residence TEXT\n        )\n    '
InterfaceError: cursor already closed
Query: b'\n        CREATE TABLE demographic_info (\n            demo_key INTEGER PRIMARY KEY,\n            ever_married TEXT,\n            work_type TEXT,\n            residence TEXT\n        )\n    '
InterfaceError: cursor already closed
Query: b'\n        CREATE TABLE demographic_info (\n            demo_key INTEGER PRIMARY KEY,\n            ever_married TEXT,\n            work_type TEXT,\n            residence TEXT\n        )\n    '
InterfaceError: cursor already closed
Query: b'\n        CREATE TABLE demographic_info (\n            demo_key INTEGER PRIMARY KEY,\n            ever_married TEXT,\n    